# Amharic Hate Speech Detector using Naive Bayes

## Preprocessing

### Importing Dataset

In [1]:
import re
import math
import pandas
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline

In [3]:
DATASET_FILEPATH = "datasets/dataset.xls"
raw_data = pandas.read_excel(DATASET_FILEPATH, sheet_name='sheet')

raw_data = shuffle(raw_data)
raw_data.head()

,Comment No.,0,Comment,Like,(view source),Timestamp
2577,NaN,0.0,Mikir Amhara ሲጀመር ፋኖ እና ኣማራ ኣንድ ኣታድርጋቸው፡ ኣማራ ት...,0,view comment,"Saturday, April 11, 2020 9:50pm GMT"
2601,23.0,0.0,ጅቡቲ ኣስመላሽ ኮሚቴ ይቋቋምልን,1,view comment,"Saturday, April 11, 2020 3:48pm GMT"
1665,NaN,0.0,Haara Bikila ዋናው እንስሳ'ማ የተሰራው ነገር ችግር እንዳለበት እ...,6,view comment,"Wednesday, April 22, 2020 11:23am GMT"
133,NaN,0.0,Elshaday Geta አንተ ፍጤ ሴትዮህን ያዝልኝ...ሆሆሆ በዚ ማታ ከአ...,0,view comment,"Tuesday, April 21, 2020 8:16pm GMT"
3398,NaN,0.0,"Yefat Yefat forget it I don't care about that,...",0,view comment,"Sunday, April 19, 2020 1:57pm GMT"


### Extracting Data

In [4]:
labels = raw_data.iloc[:, 1]
comments = raw_data.iloc[:, 2]

### Remove Alphabets

In [5]:
comments = [re.sub(r"[A-Za-z]", "", str(comment)) for comment in comments]

### Remove Numbers

In [6]:
comments = [re.sub('[0-9]', "", str(comment)) for comment in comments]

### Remove Punctuations

In [7]:
punctuations = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~፡።፣፥¡"
comments = [str(comment).translate(str.maketrans('', '', punctuations)) for comment in comments]

### Remove Whitespaces

In [8]:
comments = [(" ".join([ word for word in str(comment).split()])) for comment in comments]

### Remove Empty Comments

In [9]:
X = []
Y = []

number_of_comments = len(comments)
for i in range(number_of_comments):
    if comments[i].strip() and not math.isnan(float(labels[i])):
        X.append(comments[i])
        Y.append(labels[i])

## TF-IDF Encoding

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_features=500, min_df=5, max_df=0.7)
X = tfidf_vectorizer.fit_transform(X).toarray()

In [12]:
print("Training Dataset Shape: ", X.shape)
print("Training Label Shape: ", len(Y))

Training Dataset Shape:  (2480, 500)
Training Label Shape:  2480


## Split Dataset

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [14]:
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)

X_train Shape:  (1984, 500)
X_test Shape:  (496, 500)


In [15]:
print("Y_train Shape: ", len(Y_train))
print("Y_test Shape: ", len(Y_test))

Y_train Shape:  1984
Y_test Shape:  496


## Naive Bayes

In [16]:
from sklearn.naive_bayes import GaussianNB

In [17]:
classifier = GaussianNB()
classifier.fit(X_train, Y_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

## Evaluate Model

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [19]:
Y_pred = classifier.predict(X_test)

### Confusion Matrix

In [20]:
print(confusion_matrix(Y_test, Y_pred))

[[313 166]
 [ 10   7]]


### Classification Report

In [21]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.65      0.78       479
         1.0       0.04      0.41      0.07        17

    accuracy                           0.65       496
   macro avg       0.50      0.53      0.43       496
weighted avg       0.94      0.65      0.76       496



### Accuracy Score

In [22]:
print(accuracy_score(Y_test, Y_pred))

0.6451612903225806
